In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [4]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [5]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# KNN

In [24]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [25]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [26]:
knn = KNeighborsRegressor()
knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: knn.predict(x)[0],axis = 1)
precision = knn.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {}, error = {}".format(precision, error))

Precision = 66.47044903697412, error = 46257434434.250084


### Ahora que tenemos una intuicion, probamos variando los parametros

In [27]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [28]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [20]:
lista_k = [5,10,15,20,30,50,100,500,1000]
lista_dist = [1,2,3,4,5,50]

for k in lista_k:
    for d in lista_dist:
        knn = KNeighborsRegressor(n_neighbors = k, p = d)
        knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: knn.predict(x)[0],axis = 1)
        precision = knn.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        res.append((k,d,precision,error))
        print(k,' - ',d)

5  -  1
5  -  2
10  -  1
10  -  2
50  -  1
50  -  2
500  -  1
500  -  2


In [21]:
for r in res:
    print ("k = {}, dist = {}, precision = {}, error = {}".format(r[0],r[1],r[2],r[3]))

k = 5, dist = 1, error = 17.279051281715844, cantidad = 14846.
k = 5, dist = 2, error = 17.317874394135966, cantidad = 14843.
k = 10, dist = 1, error = 19.08697394391251, cantidad = 14807.
k = 10, dist = 2, error = 19.181302488471175, cantidad = 14801.
k = 50, dist = 1, error = 24.7305050085274, cantidad = 14707.
k = 50, dist = 2, error = 25.112657532368523, cantidad = 14699.
k = 500, dist = 1, error = 30.556412669287525, cantidad = 14576.
k = 500, dist = 2, error = 31.553976700375664, cantidad = 14550.


# Elegimos k = {}  y dist = {}

In [ ]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [ ]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [ ]:
k = 0
d = 0

In [ ]:
knn = KNeighborsRegressor(n_neighbors = k, p = d)
knn.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: knn.predict(x)[0],axis = 1)
precision = knn.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {}, error = {}".format(precision, error))

# Calculamos los verdaderos datos

In [29]:
analizar = pd.read_csv('/home/mati/Desktop/properati_dataset_modificado.csv')

In [ ]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: knn.predict(x)[0],axis = 1)

In [ ]:
analizar.to_csv('KNN_resultados.csv', index = False)